In this demo file, we investigate methods for explainability in medical imaging using pre-trained TorchXRayVision models ( https://github.com/mlmed/torchxrayvision ).

# Setup

In [1]:
!pip install torchxrayvision captum gradio
!git clone https://github.com/mlmed/gifsplanation

from pathlib import Path
from typing import Callable

import gradio as gr
import numpy as np
import torchxrayvision as xrv
import skimage, torch, torchvision
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import sys,os
sys.path.insert(0,"./gifsplanation/")

from captum.attr import IntegratedGradients, Saliency, InputXGradient
import attribution

!wget https://raw.githubusercontent.com/mlmed/torchxrayvision/master/tests/16747_3_1.jpg #download test image

fatal: destination path 'gifsplanation' already exists and is not an empty directory.


ImportError: dlopen(/opt/miniconda3/envs/RL/lib/python3.10/site-packages/_cffi_backend.cpython-310-darwin.so, 0x0002): Library not loaded: @rpath/libffi.7.dylib
  Referenced from: /opt/miniconda3/envs/RL/lib/python3.10/site-packages/_cffi_backend.cpython-310-darwin.so
  Reason: tried: '/opt/miniconda3/envs/RL/lib/libffi.7.dylib' (no such file), '/opt/miniconda3/envs/RL/lib/libffi.7.dylib' (no such file), '/opt/miniconda3/envs/RL/lib/python3.10/site-packages/../../libffi.7.dylib' (no such file), '/opt/miniconda3/envs/RL/lib/libffi.7.dylib' (no such file), '/opt/miniconda3/envs/RL/lib/libffi.7.dylib' (no such file), '/opt/miniconda3/envs/RL/lib/python3.10/site-packages/../../libffi.7.dylib' (no such file), '/opt/miniconda3/envs/RL/bin/../lib/libffi.7.dylib' (no such file), '/opt/miniconda3/envs/RL/bin/../lib/libffi.7.dylib' (no such file), '/usr/local/lib/libffi.7.dylib' (no such file), '/usr/lib/libffi.7.dylib' (no such file)

In [ ]:
def make_fig(plot_matrix):
    fig = plt.figure(figsize=(20,20))
    plt.imshow(plot_matrix, cmap=plt.cm.hot)
    #plt.title(plot_title)
    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    return fig

In [ ]:
def xrv_prepare_image(image):
    img = xrv.datasets.normalize(image, 255) # convert 8-bit image to [-1024, 1024] range
    img = img.mean(2)[None, ...] # Make single color channel
    transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])
    img = transform(img)
    img = torch.from_numpy(img)
    
    return img[None,...]

In [ ]:
def predict(image, model_choice):
    """Function that serves predictions."""
    img = xrv_prepare_image(image)
    model = xrv.models.DenseNet(weights=model_choice)
    model.eval()

    outputs = model(img)
    scores =  outputs[0].detach().numpy().astype(np.float) #conversion to np.float is needed for visualization with gr.Label
    label = dict(zip(model.pathologies, scores) )
    return label

In [ ]:
def explain_gradient(image, model_choice, target):
    """Function that serves explanations."""
    
    model = xrv.models.DenseNet(weights=model_choice)
    input = xrv_prepare_image(image)
    
    #Saliency
    saliency = Saliency(model)
    attr = saliency.attribute(input, target=model.pathologies.index(target))
    fig1 = make_fig( np.abs(attr[0,0].numpy()) )

    return fig1

def explain_input_x_gradient(image, model_choice, target):
    """Function that serves explanations."""
    
    model = xrv.models.DenseNet(weights=model_choice)
    input = xrv_prepare_image(image)
    
    #InputXGradient
    ixg = InputXGradient(model)
    attr = ixg.attribute(input, target=model.pathologies.index(target))
    fig2 = make_fig( np.abs(attr[0,0].detach().numpy()) ) 
    return fig2
    
def explain_integrated_gradients(image, model_choice, target):
    """Function that serves explanations."""
    
    model = xrv.models.DenseNet(weights=model_choice)
    input = xrv_prepare_image(image)
    
    #IntegratedGradients
    ig = IntegratedGradients(model)
    attr = ig.attribute(input, target=model.pathologies.index(target))
    fig3 = make_fig( np.abs(attr[0,0].detach().numpy()) )
    return fig3

def explain_gifsplanation(image, model_choice, target):
    """Function that serves explanations."""
    
    model = xrv.models.DenseNet(weights=model_choice)
    input = xrv_prepare_image(image)
    
    #Gifsplanation
    input.requires_grad=False
    ae = xrv.autoencoders.ResNetAE(weights="101-elastic")
    movie = attribution.generate_video(input, model, target, ae, target_filename="test", border=False, show=False,
                        ffmpeg_path="ffmpeg")

    return movie

In [ ]:
model_choices = ["densenet121-res224-all", "densenet121-res224-rsna","densenet121-res224-nih","densenet121-res224-pc","densenet121-res224-chex","densenet121-res224-mimic_nb","densenet121-res224-mimic_ch"]

target_choices = ['Atelectasis','Consolidation','Infiltration','Pneumothorax','Edema','Emphysema',
                'Fibrosis','Effusion','Pneumonia','Pleural_Thickening','Cardiomegaly','Nodule','Mass',
                'Hernia','Lung Lesion','Fracture','Lung Opacity','Enlarged Cardiomediastinum']

In [ ]:
# build a basic browser interface to a Python function
frontend = gr.Blocks()

with frontend:
  gr.Markdown(
    """
    # X-Ray Diagnosis
    Explore TorchXRayVision model predictions and data using this demo.
    """
    )
  #layout
  with gr.Tab("Prediction"):
    with gr.Row():
      with gr.Column():
        input_image = gr.Image(label="X-ray image")
        select_model = gr.Dropdown(label="Select model", choices=model_choices)
        with gr.Row():
          submit_button = gr.Button("Submit")
        gr.Examples(["16747_3_1.jpg"], inputs=input_image)

      with gr.Column():
        label = gr.Label(label="Multiclass predictions")

  submit_button.click(predict, [input_image, select_model], label) 

  with gr.Tab("Saliency maps"):

    with gr.Row():
      with gr.Column():
        select_target = gr.Dropdown(label="Select target", choices=target_choices)
        with gr.Row():
          original_image = gr.Image(label='Original', interactive=False)
          gradient_plot = gr.Plot(label="Gradient")
        with gr.Row():
          input_x_gradient_plot = gr.Plot(label='InputXGradient')
          integrated_gradients_plot = gr.Plot(label="IntegratedGradients")

  input_image.change(lambda s: s, inputs=input_image, outputs=original_image)
  select_target.change(explain_gradient, inputs=[input_image,select_model, select_target], outputs=gradient_plot)
  select_target.change(explain_input_x_gradient, inputs=[input_image,select_model, select_target], outputs=input_x_gradient_plot)
  select_target.change(explain_integrated_gradients, inputs=[input_image,select_model, select_target], outputs=integrated_gradients_plot)

  with gr.Tab("Gifsplanation"):
    with gr.Row():
      with gr.Column():
        select_target2 = gr.Dropdown(label="Select target", choices=target_choices)
        with gr.Row():
          original_image = gr.Image(label='Original', interactive=False)
          gifsplanation_vid = gr.Video(label="Gifsplanation")

  input_image.change(lambda s: s, inputs=input_image, outputs=original_image)
  select_target2.change(explain_gifsplanation, inputs=[input_image,select_model, select_target2], outputs=gifsplanation_vid)

frontend.launch(share=True, show_error=True)

In [ ]:
frontend.close()